<a href="https://colab.research.google.com/github/MonaGha/C3D/blob/master/Copy_of_SN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt


In [ ]:
def make_pairs(X_axis, Y_axis):
    pairImages = []
    pairLabels = []

    labels = np.unique(Y_axis)
    numClasses = len(labels)
    y2 = np.array(Y_axis)
    idx = [np.where(y2 == labels[i])[0] for i in range(0, numClasses)]

    for i in range(0, len(X_axis)):
 	# grab the current image and label belonging to the current
     	# iteration
         currentImage = X_axis[i]
         label = np.where(labels == y2[i])
         label =label[0][0]
     	# randomly pick an image that belongs to the *same* class
     	# label
         idxB = np.random.choice(idx[label])
         posImage = X_axis[idxB]
     	# prepare a positive pair and update the images and labels
     	# lists, respectively
         pairImages.append([currentImage, posImage])
         pairLabels.append([1])
         # grab the indices for each of the class labels *not* equal to
         # the current label and randomly pick an image corresponding
         # to a label *not* equal to the current label
         negIdx = np.where(labels != y2[i])[0]
         negIdx = np.random.choice(negIdx)
         idxC = np.random.choice(idx[negIdx])
         negImage = X_axis[idxC]
         # prepare a negative pair of images and update our lists
         pairImages.append([currentImage, negImage])
         pairLabels.append([0])

    return (np.array(pairImages), np.array(pairLabels))


In [ ]:
def euclidean_distance(vectors):
    #import tensorflow.keras.backend
 	# unpack the vectors into separate lists
    (featsA, featsB) = vectors
 	# compute the sum of squared distances between the vectors
    sumSquared = keras.backend.sum(keras.backend.square(featsA - featsB), axis=1, keepdims=True)
 	# return the euclidean distance between the vectors
    return keras.backend.sqrt(keras.backend.maximum(sumSquared, keras.backend.epsilon()))

In [ ]:
def plot_training(H, plotPath):
 	# construct a plot that plots and saves the training history
 	plt.style.use("ggplot")
 	plt.figure()
 	plt.plot(H.history["loss"], label="train_loss")
 	plt.plot(H.history["val_loss"], label="val_loss")
 	plt.plot(H.history["accuracy"], label="train_acc")
 	plt.plot(H.history["val_accuracy"], label="val_acc")
 	plt.title("Training Loss and Accuracy")
 	plt.xlabel("Epoch #")
 	plt.ylabel("Loss/Accuracy")
 	plt.legend(loc="lower left")
 	plt.savefig(plotPath)

In [ ]:
p = 'p2'
a1= '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/' + p + '/data/'
model = 'siamese_model-Base-p2-13-0.06-0.98.h5'
useBaseModel = False


In [ ]:

a= os.listdir(a1)
modelSave = '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/codes/output/' + p + '/'
# modelSave = '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/codes/output/' + p + '/'

# split (a) to have the class name and file name
class1 = []
for x in a:
    aa = x.split("_")
    b = aa[0]
    class1.append(b)

y = class1
X = a
df = pd.DataFrame({'FileName':a,
                       'id': class1})
print('X:')
print(X)
print('y:')
print(y)

df.to_csv('/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/Dataframes_of_files/df_id_ImgName_Siamese_11kHands.csv')

df= pd.read_csv(r"/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/Dataframes_of_files/df_id_ImgName_Siamese_11kHands.csv")
df = df.applymap(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

# Save the training data (X_train, y_train) to df
df_train = pd.DataFrame({'FileName':X_train,
                   'id': y_train})


    # Save the testing data (X_test, y_test) to df
df_test = pd.DataFrame({'FileName':X_test,
                   'id': y_test})

from PIL import Image
from numpy import asarray

# convert the images to arrays
X_train_images = []
for i in range(0,len(X_train)):
        # load the image
    image = Image.open(a1 + '/' + X_train[i])
    image =  image.resize((224, 224))
         # convert image to numpy array
    data = asarray(image)
    X_train_images.append(data)

X_test_images = []
for i in range(0,len(X_test)):
    # load the image
    image = Image.open(a1 + '/' + X_test[i])
    image =  image.resize((224, 224))
    # convert image to numpy array
    data = asarray(image)
    X_test_images.append(data)


(pairTrain, labelTrain) = make_pairs(X_train_images, y_train)
(pairTest, labelTest) = make_pairs(X_test_images, y_test)


    # specify the shape of the inputs for our network
IMG_SHAPE = (224, 224, 3)
    # specify the batch size and number of epochs
BATCH_SIZE = 20
EPOCHS = 30

    #BASE_OUTPUT = "output"
    # use the base output path to derive the path to the serialized
    # model along with training history plot
    # MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model.h5"])
    # PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

MODEL_PATH = '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/codes/output/' + p + '/siamese_model.h5'
PLOT_PATH = '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/codes/output/' + p + '/plot.png'


import tensorflow
from tensorflow import keras
    # import the necessary packages
    # from tensorflow.keras.models import Model
    # from tensorflow.keras.layers import Input
    # from tensorflow.keras.layers import Dense
    # from tensorflow.keras.layers import GlobalAveragePooling2D
    # from tensorflow.keras.layers import Lambda
    # from keras.callbacks import ModelCheckpoint
    # from tensorflow.keras.applications import DenseNet201
import matplotlib.pyplot as plt
imgA = keras.layers.Input(shape=IMG_SHAPE)
imgB = keras.layers.Input(shape=IMG_SHAPE)

if useBaseModel:
    featureExtractor = keras.applications.DenseNet169(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    # or include_top=True and add it manulally
    x = featureExtractor.output
    pooledOutput = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(960)(pooledOutput)
    featureExtractor = keras.models.Model(featureExtractor.input, outputs)
else:
    modelPath = '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/selectedModels/' + p + '/'
    featureExtractor = keras.models.load_model(modelPath + model)
    featureExtractor.summary()
    lastFC = featureExtractor.layers[-3]
    featureExtractor= keras.models.Model(inputs=featureExtractor.input, outputs=lastFC.output)

featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

    # finally, construct the siamese network
distance = keras.layers.Lambda(euclidean_distance)([featsA, featsB])
outputs = keras.layers.Dense(1, activation="sigmoid")(distance)
model = keras.models.Model(inputs=[imgA, imgB], outputs=outputs)
SGD = keras.optimizers.SGD(lr=0.01, nesterov=True, momentum=0.9)
Adam = keras.optimizers.Adam(lr=0.01)

EarlyStopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
LogFile = keras.callbacks.CSVLogger(modelSave + 'LogFile.csv' , separator=",", append=False)
if useBaseModel:
    checkpoint = keras.callbacks.ModelCheckpoint(modelSave + 'siamese_model-Base-'+ p +'-{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.h5',  monitor='val_accuracy', verbose=1, save_best_only=True,
            save_weights_only=True, mode='max')
else:
    checkpoint = keras.callbacks.ModelCheckpoint(modelSave + 'siamese_model-Finetunning'+ p +'-{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.h5',  monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=True, mode='max')

callbacks_list = [ EarlyStopping, checkpoint, LogFile]

model.summary()
    # compile the model
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer= SGD , 	metrics=["accuracy"])
    # train the model
print("[INFO] training model...")
history = model.fit(
 	[pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
 	validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
    callbacks=callbacks_list,
 	batch_size=BATCH_SIZE,
 	epochs=EPOCHS)

    # serialize the model to disk
print("[INFO] saving siamese model...")
model.save(modelSave + 'siamese_model.h5' )
    # plot the training history
print("[INFO] plotting training history...")
plot_training(history, PLOT_PATH)

FileNotFoundError: [Errno 2] No such file or directory: '/mmfs1/storage/users/alghamdm/Training_validation_data/L_Hand_seg/p2/data/'